In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Setup MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)  # Since we're working with images

# Dataset path
dataset_dir = "dataset"
classes = ["fall_down", "not_fallen"]

# Store landmarks and labels
X_data = []
y_data = []

# Loop through classes and images
for label, class_name in enumerate(classes):
    class_dir = os.path.join(dataset_dir, class_name)
    for img_name in tqdm(os.listdir(class_dir), desc=f"Processing {class_name}"):
        img_path = os.path.join(class_dir, img_name)
        image = cv2.imread(img_path)

        if image is None:
            continue

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            landmarks = []
            for lm in results.pose_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])  # 33 landmarks × 3 = 99
            X_data.append(landmarks)
            y_data.append(label)

# Convert to NumPy arrays
X = np.array(X_data)
y = np.array(y_data)

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape for Conv1D
x_train = x_train.reshape(x_train.shape[0], 99, 1)
x_test = x_test.reshape(x_test.shape[0], 99, 1)

# CNN model for classification
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(99, 1)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  # Binary output

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(x_train, y_train, epochs=15, batch_size=32, validation_data=(x_test, y_test))


Processing not_fallen: 100%|█████████████████████████████████████████████████████████| 492/492 [00:59<00:00,  8.26it/s]


Epoch 1/15
26/26 [==============================] - 2s 36ms/step - loss: 0.4507 - accuracy: 0.7697 - val_loss: 0.3203 - val_accuracy: 0.8725
Epoch 2/15
26/26 [==============================] - 1s 21ms/step - loss: 0.2672 - accuracy: 0.8990 - val_loss: 0.3067 - val_accuracy: 0.8676
Epoch 3/15
26/26 [==============================] - 1s 22ms/step - loss: 0.2521 - accuracy: 0.9064 - val_loss: 0.3251 - val_accuracy: 0.8676
Epoch 4/15
26/26 [==============================] - 1s 23ms/step - loss: 0.2313 - accuracy: 0.9224 - val_loss: 0.3469 - val_accuracy: 0.8676
Epoch 5/15
26/26 [==============================] - 1s 20ms/step - loss: 0.2744 - accuracy: 0.9076 - val_loss: 0.3172 - val_accuracy: 0.8725
Epoch 6/15
26/26 [==============================] - 1s 19ms/step - loss: 0.2166 - accuracy: 0.9187 - val_loss: 0.3127 - val_accuracy: 0.8922
Epoch 7/15
26/26 [==============================] - 1s 20ms/step - loss: 0.2187 - accuracy: 0.9187 - val_loss: 0.3101 - val_accuracy: 0.8873
Epoch 8/15
26

In [2]:
# Save the trained model
model.save("falldown_pose_model.h5")

In [3]:
from tensorflow.keras.models import load_model
model = load_model("falldown_pose_model.h5")

In [4]:
# Evaluate on test data
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

7/7 [==============================] - 0s 6ms/step - loss: 0.2672 - accuracy: 0.9020
Test Loss: 0.2672
Test Accuracy: 0.9020
